# __Codename: Plutus IO__

##### __Functional TODOs__:

1. Improve visualisations:
  - focus on lifetime picture
3. Add shares
  - in company
  - stocks
4. Add land
5. Add "tax account"
6. Add mortgage prepayment, fix...
7. Add concept of a family

In [1]:
# libs
import sys, logging, json
sys.path.append("/home/rado/Projects/wealthManagement/plutus/")

from context.instruments import CurrentAccount, Mortgage
from context.assets import RealEstate
from context.agreements import RentalAgreement, UtilitiesAgreement, EmployeeContract
from context.party import Party, Portfolio, setup_portfolio
from context.behaviour import transfer
from context.market import extrapolate_index, setup_market

# delete log
import os
if os.path.exists("plutus.log"): os.remove("plutus.log")

## Setup

In [2]:
# (1) High-level configuration
with open('_config_run.json') as f: config = json.load(f)
    
# loggers
logger = logging.getLogger('plutusLogger')
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s :: %(name)s | %(levelname)s | %(message)s')

fileHandler = logging.FileHandler(config['logPath'])
fileHandler.setFormatter(formatter)
logger.addHandler(fileHandler)
    
# (2) Market
with open('_config_market.json') as f: market_config = json.load(f)
indices = setup_market(market_config, config, logger)

# (3) Portfolio
with open('_config_portfolio.json') as f: portfolio_config = json.load(f)
portfolio = setup_portfolio(portfolio_config, indices, logger)

In [ ]:
# # configs for complicated Events:
# portfolio_config['employee_contracts'][0]['events'] = {'modify': [{'step':2, 'value':0}, {'step':4, 'value': 0}], 'bonus':[{'step':9, 'value':0}]}

# # dump to update versions:
# with open('_config_portfolio.json', 'w') as f: json.dump(portfolio_config, f, indent=4)

In [ ]:
# instrument-level events
mortgage_flat_events = {'cost': [{'step':4, 'value':10000}, {'step': 5, 'value': 270000}], 'income':{}}
employee_contract_events = 

# party-level events
# rado_events = {'sell': [{'step':6, 'id':'rado_mortgage_flat'}]
#              , 'buy': [{'step':8, 'id':'rado_flat3', 'type':'RealEstate', 'current_market_value':3e6, 'property_tax':2000, 'house_community_costs':6000, 'real_estate_index':re_index}]
#              , 'transfer': [{'step':10, 'from':'rado_current_account', 'to':'rado_saving_account','amount':1e6}]}

rado_events = {'transfer': transfer(amount=25000, a='rado_current_account', b='rado_saving_account', n_years=config['n_years'], frequency=1)}
# rado_events = {}

#### [4] Portfolio

In [ ]:
# Income
employee_contract = EmployeeContract(id='rado_credo', salary = 100000, income_tax_rate = 0.0, logger=logger, events=employee_contract_events, n_years=4)

# SA & CA
current_account = CurrentAccount(id='rado_current_account', current_outstanding=0.0, monthly_cost= 0.0, cnit=0.0, logger=logger, primary=True)
saving_account = CurrentAccount(id='rado_saving_account', current_outstanding=0.0, monthly_cost= 0.0, cnit=0.001, logger=logger)

# Shares


# Real Estate
inherited_flat = RealEstate(id='rado_inherited_flat', current_market_value=1.8e6, property_tax = 2000, house_community_costs = 5000, real_estate_index = re_index, logger=logger, events=None) 
# mortgage = Mortgage(id='rado_mortgage', principal=3e6, cnit=0.02, maturity_in_years=30, logger=logger)
# mortgage_flat = RealEstate(id='rado_mortgage_flat', current_market_value=5e6, property_tax = 2000, house_community_costs = 6000, real_estate_index = re_index, logger=logger, events=mortgage_flat_events)

# Rental agreement
rent_from_inherited_flat = RentalAgreement(id='rado_rent_from_inherited_flat', logger=logger, inflation_index=inflation_index, my_position='renter', rent=10000, yearly_costs=0)

#### [5] Parties

In [ ]:
rado = Party(id='Rado'
             , logger=logger 
             , initial_free_cash=0
             , initial_portfolio=Portfolio(logger, 
                                           current_account, 
                                           saving_account, 
                                           inherited_flat,
                                           rent_from_inherited_flat,
                                           # mortgage, 
                                           # mortgage_flat, 
                                           employee_contract)
             , monthly_expenditures=50000
             , discount_rate=0.015
             , events=rado_events)

## __Simulation__

In [ ]:
logger.info('')
logger.info('============================================')
logger.info('============== SIMULATION  =================')
logger.info('============================================')
logger.info('')

In [ ]:
rado.live(config['n_years']*12+1)

In [ ]:
# plots
from core.plots import PartyPlotter

plt = PartyPlotter(rado)

### Plots

In [ ]:
#plt.plot_position_evolution()

In [ ]:
plt.plot_wealth_evolution()